In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sympy as sym
import autograd.numpy as np
from autograd import grad
from autograd import hessian
from IPython.display import Markdown, display
import copy
from matplotlib import gridspec
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import sklearn.datasets
from skimage.feature import hog
from skimage import data, exposure
import cv2
import os

In [ ]:
# Path to the folder containing images
train_path = "/content/drive/MyDrive/EE_Final_Project/train"
class_labels = ["angry", "disgusted", "fearful", "happy", "neutral", "sad", "surprised"]

# List to store images and corresponding file names
images = []
image_names = []

# Iterate through all files in the folder
for class_label in class_labels:
    image_count = 0

    folder_path = os.path.join(train_path, class_label)

    # Create a list to store the loaded images for the current class
    current_class_list = []
    for filename in os.listdir(folder_path):

        # Check if the file is a supported image file
        if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Construct the full file path
            file_path = os.path.join(folder_path, filename)

            # Read the image using OpenCV
            img = plt.imread(file_path)

            # Append the image and its name to the lists
            current_class_list.append(img)
            image_count+=1
    globals()["train_" + class_label] = current_class_list

In [ ]:
# Path to the folder containing images
test_path = "/content/drive/MyDrive/EE_Final_Project/test"
class_labels = ["angry", "disgusted", "fearful", "happy", "neutral", "sad", "surprised"]

# List to store images and corresponding file names
images = []
image_names = []

# Iterate through all files in the folder
for class_label in class_labels:

    folder_path = os.path.join(test_path, class_label)

    # Create a list to store the loaded images for the current class
    current_class_list = []
    for filename in os.listdir(folder_path):
        # if image_count > 1000:
        #   break
        # Check if the file is a supported image file
        if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Construct the full file path
            file_path = os.path.join(folder_path, filename)

            # Read the image using OpenCV
            img = plt.imread(file_path)

            # Append the image and its name to the lists
            current_class_list.append(img)
    globals()["test_" + class_label] = current_class_list

In [ ]:
#train
x_angry = ([])
for image in train_angry:
    image = np.array(image) / 255
    x_angry.append(image)

x_disgusted = ([])
for image in train_disgusted:
    image = np.array(image) / 255
    x_disgusted.append(image)

x_fearful = ([])
for image in train_fearful:
    image = np.array(image) / 255
    x_fearful.append(image)

x_happy = ([])
for image in train_happy:
    image = np.array(image) / 255
    x_happy.append(image)

x_neutral = ([])
for image in train_neutral:
    image = np.array(image) / 255
    x_neutral.append(image)

x_sad = ([])
for image in train_sad:
    image = np.array(image) / 255
    x_sad.append(image)

x_surprised = ([])
for image in train_surprised:
    image = np.array(image) / 255
    x_surprised.append(image)


# test
x_angry_test = ([])
for image in test_angry:
    image = np.array(image) / 255
    x_angry_test.append(image)

x_disgusted_test = ([])
for image in test_disgusted:
    image = np.array(image) / 255
    x_disgusted_test.append(image)

x_fearful_test = ([])
for image in test_fearful:
    image = np.array(image) / 255
    x_fearful_test.append(image)

x_happy_test = ([])
for image in test_happy:
    image = np.array(image) / 255
    x_happy_test.append(image)

x_neutral_test = ([])
for image in test_neutral:
    image = np.array(image) / 255
    x_neutral_test.append(image)

x_sad_test = ([])
for image in test_sad:
    image = np.array(image) / 255
    x_sad_test.append(image)

x_surprised_test = ([])
for image in test_surprised:
    image = np.array(image) / 255
    x_surprised_test.append(image)

In [ ]:
# train data
angry_features_list = []
disgusted_features_list = []
fearful_features_list = []
happy_features_list = []
neutral_features_list = []
sad_features_list = []
surprised_features_list = []

feature = [x_angry,x_disgusted,x_fearful,x_happy,x_neutral,x_sad,x_surprised]
feature_list_train = [angry_features_list, disgusted_features_list, fearful_features_list, happy_features_list, neutral_features_list, sad_features_list, surprised_features_list]

for i in range(len(feature)):
  for image_array in feature[i]:
      features, hog_image = hog(
          image_array,
          orientations=8,
          pixels_per_cell=(8, 8),
          cells_per_block=(1, 1),
          visualize=True,
      )
      feature_list_train[i].append(features)

angry_features_list_test = []
disgusted_features_list_test = []
fearful_features_list_test = []
happy_features_list_test = []
neutral_features_list_test = []
sad_features_list_test = []
surprised_features_list_test = []

feature = [x_angry_test,x_disgusted_test,x_fearful_test,x_happy_test,x_neutral_test,x_sad_test,x_surprised_test]
feature_list_test = [angry_features_list_test,
                     disgusted_features_list_test,
                     fearful_features_list_test,
                     happy_features_list_test,
                     neutral_features_list_test,
                     sad_features_list_test,
                     surprised_features_list_test]

for i in range(len(feature)):
  for image_array in feature[i]:
      features, hog_image = hog(
          image_array,
          orientations=8,
          pixels_per_cell=(8, 8),
          cells_per_block=(1, 1),
          visualize=True,
      )
      feature_list_test[i].append(features)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score

x_train = np.concatenate(
    (
        feature_list_train[0],
        feature_list_train[1],
        feature_list_train[2],
        feature_list_train[3],
        feature_list_train[4],
        feature_list_train[5],
        feature_list_train[6],
    ),
    axis=0,
)
y_train = (
    [0] * len(feature_list_train[0])
    + [1] * len(feature_list_train[1])
    + [2] * len(feature_list_train[2])
    + [3] * len(feature_list_train[3])
    + [4] * len(feature_list_train[4])
    + [5] * len(feature_list_train[5])
    + [6] * len(feature_list_train[6])
)
clf = SVC(kernel='rbf')

# Specify the number of folds (k)
num_folds = 5  # You can adjust this value based on your needs

# Create a KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation
accuracy_scores = cross_val_score(clf, x_train, y_train, cv=kf, scoring='accuracy')

# Print the accuracy scores for each fold
for fold, accuracy in enumerate(accuracy_scores, 1):
    print(f'Fold {fold}: Accuracy = {accuracy:.4f}')

# Print the mean accuracy across all folds
mean_accuracy = np.mean(accuracy_scores)
print(f'\nMean Accuracy: {mean_accuracy:.4f}')

clf.fit(x_train, y_train)

# Grid Search for Hyperparameter Tuning:
# Perform a grid search over a range of hyperparameter values to find the combination that yields the best performance.
# param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf']}
# grid_search = GridSearchCV(SVC(), param_grid, cv=5)
# grid_search.fit(x_train, y_train)
# best_params = grid_search.best_params_

x_test = np.concatenate(
    (
        feature_list_test[0],
        feature_list_test[1],
        feature_list_test[2],
        feature_list_test[3],
        feature_list_test[4],
        feature_list_test[5],
        feature_list_test[6],
    ),
    axis=0,
)
y_test = (
    [0] * len(feature_list_test[0])
    + [1] * len(feature_list_test[1])
    + [2] * len(feature_list_test[2])
    + [3] * len(feature_list_test[3])
    + [4] * len(feature_list_test[4])
    + [5] * len(feature_list_test[5])
    + [6] * len(feature_list_test[6])
)

# X_test: Feature vectors for testing images
# y_test: True labels for testing images

# accuracy = clf.score(x_test, y_test)

y_pred = clf.predict(x_test)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print(f'\nTest Accuracy: {test_accuracy:.4f}')
# accuracy = grid_search.score(x_test, y_test)

Fold 1: Accuracy = 0.4688
Fold 2: Accuracy = 0.4662
Fold 3: Accuracy = 0.4577
Fold 4: Accuracy = 0.4756
Fold 5: Accuracy = 0.4522

Mean Accuracy: 0.4641

Test Accuracy: 0.4755
